# DDPM forward and reverse process implementation
Paper: https://arxiv.org/abs/2006.11239

## Forward Process
Fowrad process adss noise to the data


In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
import matplotlib.pyplot as plt
from tqdm import tqdm
import os
import time
from data.dataset import BEVFeaturesDataset, PaddDataset
from torch.utils.data import DataLoader, Dataset
import wandb
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [2]:
!CUDA_LAUNCH_BLOCKING=1

In [3]:
import math
import torch
from torch import nn
from inspect import isfunction

class UNet(nn.Module):
    def __init__(
        self,
        in_channel=6,
        out_channel=3,
        inner_channel=32,
        norm_groups=32,
        channel_mults=(1, 2, 4, 8, 8),
        attn_res=(8),
        res_blocks=3,
        dropout=0,
        with_noise_level_emb=True,
        image_size=128,
        eps=1e-5
    ):
        super().__init__()

        if with_noise_level_emb:
            noise_level_channel = inner_channel
            self.noise_level_mlp = nn.Sequential(
                PositionalEncoding(inner_channel),
                nn.Linear(inner_channel, inner_channel * 4),
                Swish(),
                nn.Linear(inner_channel * 4, inner_channel)
            )
        else:
            noise_level_channel = None
            self.noise_level_mlp = None
        
        self.image_size = image_size
        self.in_channel = in_channel
        self.out_channel = out_channel

        num_mults = len(channel_mults)
        pre_channel = inner_channel
        feat_channels = [pre_channel]
        now_res = image_size
        downs = [nn.Conv2d(in_channel, inner_channel,
                           kernel_size=3, padding=1)]
        for ind in range(num_mults):
            is_last = (ind == num_mults - 1)
            use_attn = (now_res in attn_res)
            channel_mult = inner_channel * channel_mults[ind]
            for _ in range(0, res_blocks):
                downs.append(ResnetBlocWithAttn(
                    pre_channel, channel_mult, noise_level_emb_dim=noise_level_channel, norm_groups=norm_groups, dropout=dropout, with_attn=use_attn, eps=eps))
                feat_channels.append(channel_mult)
                pre_channel = channel_mult
            if not is_last:
                downs.append(Downsample(pre_channel))
                feat_channels.append(pre_channel)
                now_res = now_res//2
        self.downs = nn.ModuleList(downs)

        self.mid = nn.ModuleList([
            ResnetBlocWithAttn(pre_channel, pre_channel, noise_level_emb_dim=noise_level_channel, norm_groups=norm_groups,
                               dropout=dropout, with_attn=True, eps=eps),
            ResnetBlocWithAttn(pre_channel, pre_channel, noise_level_emb_dim=noise_level_channel, norm_groups=norm_groups,
                               dropout=dropout, with_attn=False, eps=eps)
        ])

        ups = []
        for ind in reversed(range(num_mults)):
            is_last = (ind < 1)
            use_attn = (now_res in attn_res)
            channel_mult = inner_channel * channel_mults[ind]
            for _ in range(0, res_blocks+1):
                ups.append(ResnetBlocWithAttn(
                    pre_channel+feat_channels.pop(), channel_mult, noise_level_emb_dim=noise_level_channel, norm_groups=norm_groups,
                        dropout=dropout, with_attn=use_attn, eps=eps))
                pre_channel = channel_mult
            if not is_last:
                ups.append(Upsample(pre_channel))
                now_res = now_res*2

        self.ups = nn.ModuleList(ups)

        self.final_conv = Block(pre_channel, default(out_channel, in_channel), groups=norm_groups, eps=eps)

    def forward(self, x, time):
        t = self.noise_level_mlp(time) if exists(
            self.noise_level_mlp) else None

        feats = []
        for layer in self.downs:
            if isinstance(layer, ResnetBlocWithAttn):
                x = layer(x, t)
            else:
                x = layer(x)
            feats.append(x)

        for layer in self.mid:
            if isinstance(layer, ResnetBlocWithAttn):
                x = layer(x, t)
            else:
                x = layer(x)

        for layer in self.ups:
            if isinstance(layer, ResnetBlocWithAttn):
                x = layer(torch.cat((x, feats.pop()), dim=1), t)
            else:
                x = layer(x)

        return self.final_conv(x)


# PositionalEncoding Source： https://github.com/lmnt-com/wavegrad/blob/master/src/wavegrad/model.py
class PositionalEncoding(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.dim = dim

    def forward(self, noise_level):
        count = self.dim // 2
        step = torch.arange(count, dtype=noise_level.dtype, device=noise_level.device) / count
        encoding = noise_level.unsqueeze(1) * torch.exp(-math.log(1e4) * step.unsqueeze(0))
        encoding = torch.cat([torch.sin(encoding), torch.cos(encoding)], dim=-1)
        return encoding


class FeatureWiseAffine(nn.Module):
    def __init__(self, in_channels, out_channels, use_affine_level=False):
        super(FeatureWiseAffine, self).__init__()
        self.use_affine_level = use_affine_level
        self.noise_func = nn.Sequential(
            nn.Linear(in_channels, out_channels*(1+self.use_affine_level))
        )

    def forward(self, x, noise_embed):
        batch = x.shape[0]
        if self.use_affine_level:
            gamma, beta = self.noise_func(noise_embed).view(batch, -1, 1, 1).chunk(2, dim=1)
            x = (1 + gamma) * x + beta
        else:
            x = x + self.noise_func(noise_embed).view(batch, -1, 1, 1)
        return x


class Swish(nn.Module):
    def forward(self, x):
        return x * torch.sigmoid(x)


class Upsample(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.up = nn.Upsample(scale_factor=2, mode="nearest")
        self.conv = nn.Conv2d(dim, dim, 3, padding=1)

    def forward(self, x):
        return self.conv(self.up(x))


class Downsample(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.conv = nn.Conv2d(dim, dim, 3, 2, 1)

    def forward(self, x):
        return self.conv(x)


# building block modules


class Block(nn.Module):
    def __init__(self, dim, dim_out, groups=32, dropout=0, eps=1e-5):
        super().__init__()
        self.block = nn.Sequential(
            nn.GroupNorm(groups, dim, eps=eps),
            Swish(),
            nn.Dropout(dropout) if dropout != 0 else nn.Identity(),
            nn.Conv2d(dim, dim_out, 3, padding=1)
        )

    def forward(self, x):
        return self.block(x)


class ResnetBlock(nn.Module):
    def __init__(self, dim, dim_out, noise_level_emb_dim=None, dropout=0, use_affine_level=False, norm_groups=32, eps=1e-5):
        super().__init__()
        self.noise_func = FeatureWiseAffine(noise_level_emb_dim, dim_out, use_affine_level)

        self.block1 = Block(dim, dim_out, groups=norm_groups, eps=eps)
        self.block2 = Block(dim_out, dim_out, groups=norm_groups, dropout=dropout, eps=eps)
        self.res_conv = nn.Conv2d(
            dim, dim_out, 1) if dim != dim_out else nn.Identity()

    def forward(self, x, time_emb):
        b, c, h, w = x.shape
        h = self.block1(x)
        h = self.noise_func(h, time_emb)
        h = self.block2(h)
        return h + self.res_conv(x)


class SelfAttention(nn.Module):
    def __init__(self, in_channel, n_head=1, norm_groups=32, eps=1e-5):
        super().__init__()

        self.n_head = n_head

        self.norm = nn.GroupNorm(norm_groups, in_channel, eps=eps)
        self.qkv = nn.Conv2d(in_channel, in_channel * 3, 1, bias=False)
        self.out = nn.Conv2d(in_channel, in_channel, 1)

    def forward(self, input):
        batch, channel, height, width = input.shape
        n_head = self.n_head
        head_dim = channel // n_head

        norm = self.norm(input)
        qkv = self.qkv(norm).view(batch, n_head, head_dim * 3, height, width)
        query, key, value = qkv.chunk(3, dim=2)  # bhdyx

        attn = torch.einsum("bnchw, bncyx -> bnhwyx", query, key).contiguous() / math.sqrt(channel)
        attn = attn.view(batch, n_head, height, width, -1)
        attn = torch.softmax(attn, -1)
        attn = attn.view(batch, n_head, height, width, height, width)

        out = torch.einsum("bnhwyx, bncyx -> bnchw", attn, value).contiguous()
        out = self.out(out.view(batch, channel, height, width))

        return out + input


class ResnetBlocWithAttn(nn.Module):
    def __init__(self, dim, dim_out, *, noise_level_emb_dim=None, norm_groups=32, dropout=0, with_attn=False, eps=1e-5):
        super().__init__()
        self.with_attn = with_attn
        self.res_block = ResnetBlock(
            dim, dim_out, noise_level_emb_dim, norm_groups=norm_groups, dropout=dropout, eps=eps)
        if with_attn:
            self.attn = SelfAttention(dim_out, norm_groups=norm_groups, eps=eps)

    def forward(self, x, time_emb):
        x = self.res_block(x, time_emb)
        if(self.with_attn):
            x = self.attn(x)
        return x


def exists(x):
    return x is not None


def default(val, d):
    if exists(val):
        return val
    return d() if isfunction(d) else d

In [ ]:
from numpy import mean, var
from functools import partial


def _warmup_beta(linear_start, linear_end, n_timestep, warmup_frac):
    betas = linear_end * np.ones(n_timestep, dtype=np.float64)
    warmup_time = int(n_timestep * warmup_frac)
    betas[:warmup_time] = np.linspace(
        linear_start, linear_end, warmup_time, dtype=np.float64)
    return betas

def make_beta_schedule(schedule, n_timestep, linear_start=1e-6, linear_end=1e-2, cosine_s=8e-3):
    """
    Create a beta schedule that is a function of the number of diffusion steps.
    Return:
        betas: a numpy array of shape (n_timestep,) that defines the beta schedule
    """
    if schedule == 'quad':
        betas = np.linspace(linear_start ** 0.5, linear_end ** 0.5,
                            n_timestep) ** 2
    elif schedule == 'linear':
        betas = np.linspace(linear_start, linear_end,
                            n_timestep)
    elif schedule == 'warmup10':
        betas = _warmup_beta(linear_start, linear_end,
                             n_timestep, 0.1)
    elif schedule == 'warmup50':
        betas = _warmup_beta(linear_start, linear_end,
                             n_timestep, 0.5)
    elif schedule == 'const':
        betas = linear_end * np.ones(n_timestep, dtype=np.float64)
    elif schedule == 'jsd':  # 1/T, 1/(T-1), 1/(T-2), ..., 1
        betas = 1. / np.linspace(n_timestep,
                                 1, n_timestep, dtype=np.float64)
    elif schedule == "cosine":
        timesteps = (
            torch.arange(n_timestep + 1, dtype=torch.float64) /
            n_timestep + cosine_s
        )
        alphas = timesteps / (1 + cosine_s) * math.pi / 2
        alphas = torch.cos(alphas).pow(2)
        alphas = alphas / alphas[0]
        betas = 1 - alphas[1:] / alphas[:-1]
        betas = betas.clamp(max=0.999)
    else:
        raise NotImplementedError(schedule)
    return torch.from_numpy(betas) if betas.type == np.ndarray else betas


class DenoiseDiffusion(nn.Module):
    def __init__(self, eps_model, beta_schedule, loss_fn=nn.L1Loss()):
        super().__init__()
        # Parameters for training
        self.loss_fn = loss_fn
        self.eps_model = eps_model
        self.beta_schedule = beta_schedule


        # Parameters for diffusion process         
    def set_new_noise_schedule(self, device=torch.device('cuda'), phase='train'):
        self.n_steps = self.beta_schedule[phase]['n_timestep']
        to_torch = partial(torch.as_tensor, dtype=torch.float32, device=device)

        betas = make_beta_schedule(**self.beta_schedule[phase])
        # self.betas = beta.type(dtype=torch.float32).to(self.eps_model.device)
        alphas = 1. - betas
        gammas = torch.cumprod(alphas, dim=0)
        sigmas = torch.sqrt(1.0 - torch.pow(alphas, 2))
        lambdas = torch.log(alphas / sigmas)

        self.register_buffer("betas", to_torch(betas))
        self.register_buffer("alphas", to_torch(alphas))
        self.register_buffer("gammas", to_torch(gammas))
        self.register_buffer("sigmas", to_torch(sigmas))
        self.register_buffer("lambdas", to_torch(lambdas))


    # def to(self, device):
        # self.eps_model = self.eps_model.to(device)

    def gather(self, tensor, t):
        """
        Gather the values of x at the time steps t.
        Makes it compatible with the shape of x0, which is (B, C, H, W).
        Args:
            tensor: a tensor of shape (n_steps,)
            t: a tensor of shape (B,)
        Return:
            a tensor of shape (B, 1, 1, 1) that contains the values of x at the time steps t
        """
        t = tensor.gather(-1, t)
        return t.reshape(-1, 1, 1, 1)
    
        # We need a function that samples the batch 
    def q_sample(self, y0, sample_gammas, noise=None):
        """
        Sample from q(yt|y0), reading same as sample xt at step t given x0.
        Other implementations also use function q_xt_x0 first but we can directly implement it here.
        Args:
            y0: the original data, shape (B, C, H, W)
            sample_gammas: the gamma values for sampling, shape (B,)
            noise: the noise, shape (B, C, H, W)
        Return:
            yt: the noisy data at time step t, shape (B, C, H, W)
        """
        eps = torch.randn_like(y0, device=y0.device) if noise is None else noise
        
        return (
            torch.sqrt(sample_gammas) * y0 + torch.sqrt(1 - sample_gammas) * eps
        )
    

    def forward(self, y0, y_cond=None):
        """
        Algorithm 1 in Denoising Diffusion Probalisitic Models

        Args:
            y0: the original data, shape (B, C, H, W)
        """
        b, *_ = y0.shape

        t = torch.randint(1, self.n_steps, (b,), device=y0.device, dtype=torch.long)
        # Select a random gamma for each sample in the batch, which is between gamma_t and gamma_t-1 of generated timesteps t. This is to make the training more stable and avoid overfitting to specific timesteps.
        gamma_t1 = self.gather(self.gammas, t - 1)
        gamma_t2 = self.gather(self.gammas, t)
        sample_gammas = (gamma_t2 - gamma_t1) * torch.rand((b, 1, 1, 1), device=y0.device) + gamma_t1
        sample_gammas = sample_gammas

        # Create the noise to compare it to the predicted noise, which is used for training the model. This is the noise added to the original data to get the noisy data at time step t.
        noise = torch.randn_like(y0, device=y0.device)
        y_noisy = self.q_sample(y0, sample_gammas, noise=noise)

        noise_hat = self.eps_model(torch.cat([y_noisy, y_cond], dim=1) if y_cond is not None else y_noisy, sample_gammas)

        loss = self.loss_fn(noise_hat, noise)
        return loss
    

    # Samplers

    @torch.no_grad()
    #TODO implement y_cond and remove n_samples
    def ddpm_sampler(self, y_cond=None, sample_inter=10, clip_denoised=True):
        """
        https://arxiv.org/pdf/2006.11239
        Implementation of algorithm 2. However, to keep sampling stable we calculate the start from noise, clamp it and use the posterior of equation 7 to calculate y_t-1.
        We use equation 15 to calculate y_0 (start from noise), then clamp it. Then we use equation 7 to calculate y_t-1 = mean + sigma * z
        """
        y = torch.randn_like(y_cond, device=y_cond.device)
        ret_arr = y.clone()
        for i in tqdm(reversed(range(self.n_steps)), desc='DDPM sampler', total=self.n_steps):
            z = torch.randn_like(y) if i > 1 else torch.zeros_like(y)   
            t_tensor = torch.full((y_cond.shape[0],), i, device=y_cond.device, dtype=torch.long)

            gamma_t = self.gather(self.gammas, t_tensor)
            gamma_t_prev = self.gather(self.gammas, t_tensor - 1) if i > 0 else torch.ones_like(gamma_t)
            beta_t = self.gather(self.betas, t_tensor)
            alpha_t = self.gather(self.alphas, t_tensor)

            y_0_tilde = (y - torch.sqrt(1-gamma_t)*self.eps_model(torch.cat([y, y_cond], dim=1) if y_cond is not None else y, gamma_t)) / torch.sqrt(gamma_t) # predict start

            if clip_denoised:
                y_0_tilde = torch.clamp(y_0_tilde, -1., 1.)

            # eta = (y - torch.sqrt(alpha_t)*y_0_tilde) / torch.sqrt(1-alpha_t)

            # Calculate posterior mean and variance
            mean = (torch.sqrt(gamma_t_prev) * beta_t * y_0_tilde) / (1 - gamma_t) + (torch.sqrt(alpha_t) * (1 - gamma_t_prev) * y) / (1 - gamma_t)

            sigma = (1 - gamma_t_prev) * beta_t / (1 - gamma_t)

            y = mean + sigma * z

            if i % sample_inter == 0:
                ret_arr = torch.cat((ret_arr, y), dim=0)

        return y, ret_arr
    

    @torch.no_grad()
    def ddim_sampler(self, y_cond=None, noise=None, sample_inter=10, steps=50, clip_denoised=True, eta=0.0):
        """
        DDIM sampler from https://arxiv.org/abs/2010.02502
        With eta=0, it becomes a deterministic sampler, which is the one we will use in this implementation. With eta>0, it becomes a stochastic sampler, which is similar to the DDPM sampler but with different noise scale. 
        """
        y = torch.randn_like(y_cond, device=y_cond.device) if noise is None else noise
        ret_arr = y.clone()
        step_size = self.n_steps // steps

        for i in tqdm(range(steps), desc='DDIM sampling loop timestep', total=steps):
            t = self.n_steps - i * step_size
            t_tensor = torch.full((y_cond.shape[0],), t, dtype=torch.long, device=y_cond.device)

            gamma = self.gather(self.gammas, t_tensor - 1)
            
            # Make sure that when t_tensor - step_size - 1 is negative, we use gamma_prev = 1, which means that we are at the final step and we should not add any noise.
            gamma_prev = self.gather(self.gammas, torch.clamp(t_tensor - step_size - 1, min=0)) if (t_tensor - step_size - 1 >= 0).any() else torch.ones_like(gamma)
            noise_pred = self.eps_model(torch.cat([y, y_cond], dim=1) if y_cond is not None else y, gamma)

            y0_pred = (y - torch.sqrt(1 - gamma) * noise_pred) / torch.sqrt(gamma)
            
            # Clamp prediction to stablize sampling
            if clip_denoised:
                y0_pred = torch.clamp(y0_pred, -1., 1.)

            sigma_t = eta * torch.sqrt((1 - gamma_prev) / (1 - gamma)) * torch.sqrt(1-gamma / gamma_prev)

            dir_yt = torch.sqrt(1 - gamma_prev - torch.pow(sigma_t, 2)) * noise_pred
            
            
            y = torch.sqrt(gamma_prev) * y0_pred + dir_yt + sigma_t * torch.randn_like(y)

        return y, ret_arr
    

    @torch.no_grad()
    def dpm_solver_multi_step_sampler(self, y_cond=None, sample_inter=10, steps=10, clip_denoised=True):
        """
        Implement multistep from https://arxiv.org/pdf/2211.01095
        """
        step_size = self.n_steps // steps


        yT = torch.randn_like(y_cond, device=y_cond.device)
        ret_arr = yT.clone()

        ytilde = yT


        t_0 = torch.full((y_cond.shape[0],), self.n_steps - 1, device=y_cond.device, dtype=torch.long)
        t_1 = torch.full((y_cond.shape[0],), self.n_steps - step_size - 1, device=y_cond.device, dtype=torch.long)
        t_2 = torch.full((y_cond.shape[0],), self.n_steps - 2 * step_size - 1, device=y_cond.device, dtype=torch.long)
        # print(self.lambdas.shape)
        # print(t_1)
        # print(t_0)
        h_i_prev = self.gather(self.lambdas, t_1) - self.gather(self.lambdas, t_0)

        # Buffer P and Q for multi_step sampling. P = -2 and Q = -1 in timestepe space, which means that they are the data prediction at t_i-2 and t_i-1 respectively. We will update them in each step and use them to calculate the data prediction at t_i.
        P = self.data_prediction(ytilde, y_cond=y_cond, t=t_0) # y_theta_0
        if clip_denoised:
            P = torch.clamp(P, -1., 1.)
        y_tilde = (self.gather(self.sigmas, t_1)/self.gather(self.sigmas, t_0)) * ytilde - self.gather(self.alphas, t_1) * (torch.exp(-h_i_prev) - 1) * P
        Q = self.data_prediction(y_tilde, y_cond=y_cond, t=t_1) # y_theta_2
        if clip_denoised:
            Q = torch.clamp(Q, -1, 1.)
        

        for i in tqdm((range(2, steps)), desc='DPM-Solver++(2M) sampler', initial=2, total=steps):
            t_cur = self.n_steps - i * step_size - 1
            t_prev = self.n_steps - (i - 1) * step_size - 1

            t_prev_tensor = torch.full((y_cond.shape[0],), t_prev, device=y_cond.device, dtype=torch.long)
            t_cur_tensor = torch.full((y_cond.shape[0],), t_cur, device=y_cond.device, dtype=torch.long)

            h_i_cur = self.gather(self.lambdas, t_cur_tensor) - self.gather(self.lambdas, t_prev_tensor)
            r_i = h_i_prev / h_i_cur

            D_i = (1 + 1 / (2 * r_i)) * Q - 1 / (2 * r_i) * P

            y_tilde = (self.gather(self.sigmas, t_cur_tensor) / self.gather(self.sigmas, t_prev_tensor)) * y_tilde - self.gather(self.alphas, t_cur_tensor) * (torch.exp(-h_i_cur) - 1) * D_i
            
            h_i_prev = h_i_cur
            P = Q.clone()
            if i < self.n_steps:
                Q = self.data_prediction(y_tilde, y_cond=y_cond, t=t_cur_tensor)
                if clip_denoised:
                    Q = torch.clamp(Q, -1., 1.)

            if i % sample_inter == 0:
                ret_arr = torch.cat((ret_arr, y_tilde), dim=0)

        return y_tilde, ret_arr
    
    def data_prediction(self, yt, y_cond=None, t=None):
        gamma = self.gather(self.gammas, t).to(yt.device)
        noise_pred = self.eps_model(torch.cat([yt, y_cond], dim=1) if y_cond is not None else yt, gamma)
        y0_hat = (yt - torch.sqrt(1 - gamma) * noise_pred) / torch.sqrt(gamma)

        return y0_hat

In [15]:
beta_schedule = dict(
    train=dict(
        schedule='cosine',
        n_timestep=2000,
        cosine_s=8e-3,
    ),
    test=dict(
        schedule='linear',
        n_timestep=1000,
        linear_start=1e-5,
        linear_end=1e-1,
    )
)

model_config = dict(
    in_channel=512,
    out_channel=256,
    inner_channel=128,
    norm_groups=32,
    channel_mults=(1, 2, 4, 8),
    attn_res=(25,),
    res_blocks=2,
    dropout=0,
    with_noise_level_emb=True,
    image_size=200,
    eps=1e-5
)

hyperparameters = dict(
    model_config=model_config,
    beta_schedule=beta_schedule,
    batch_size=8,
)

Unet = UNet(**model_config)
diffusion = DenoiseDiffusion(Unet, beta_schedule)
diffusion.set_new_noise_schedule(phase='train')

In [ ]:
diffusion.to('cuda')
out, _ = diffusion.ddpm_sampler(y_cond=torch.randn(1, 256, 200, 200, device='cuda'))

DDPM sampler:   0%|          | 0/2000 [00:00<?, ?it/s]

DDPM sampler:  78%|███████▊  | 1566/2000 [03:02<00:48,  9.03it/s]

In [10]:
from torchinfo import summary

summary(Unet, input_size=[(hyperparameters['batch_size'], model_config['in_channel'], model_config['image_size'], model_config['image_size']), (hyperparameters['batch_size'], 1)])

Layer (type:depth-idx)                             Output Shape              Param #
UNet                                               [8, 256, 200, 200]        --
├─Sequential: 1-1                                  [8, 1, 128]               --
│    └─PositionalEncoding: 2-1                     [8, 1, 128]               --
│    └─Linear: 2-2                                 [8, 1, 512]               66,048
│    └─Swish: 2-3                                  [8, 1, 512]               --
│    └─Linear: 2-4                                 [8, 1, 128]               65,664
├─ModuleList: 1-2                                  --                        --
│    └─Conv2d: 2-5                                 [8, 128, 200, 200]        589,952
│    └─ResnetBlocWithAttn: 2-6                     [8, 128, 200, 200]        --
│    │    └─ResnetBlock: 3-1                       [8, 128, 200, 200]        312,192
│    └─ResnetBlocWithAttn: 2-7                     [8, 128, 200, 200]        --
│    │    └─Resne

In [10]:
size = 64
square_size = 16
tensor = torch.zeros(1, 1, size, size)
start = (size - square_size) // 2
end = start + square_size
tensor[:, :, start:end, start:end] = 1.0

samples = tensor.repeat(100, 1, 1, 1)

In [ ]:
diffusion.set_new_noise_schedule(phase='test')
sampled = diffusion.dpm_solver_multi_step_sampler(1, steps=50, clip_denoised=True)
y, ret_arr = sampled
print(y)
print(f"Max: {torch.amax(y)}, Min: {torch.amin(y)}")
plt.imshow(y.cpu().numpy()[0, 0], cmap='viridis')
plt.colorbar()

In [7]:
from torch.utils.data import DataLoader

def load_data():
    # Load the saved data
    dataset = BEVFeaturesDataset(root_dir='/home/mingdayang/FeatureBridgeMapping/data/bev_features', transform=None)

    return dataset

def create_splits(dataset, train_split=0.8):
    train_dataset, test_dataset = torch.utils.data.random_split(dataset, [int(train_split * len(dataset)), len(dataset) - int(train_split * len(dataset))])

    return train_dataset, test_dataset

def make_loader(batch_size, dataset):

    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
        # Let's check out what we've created

    return dataloader

dataset = load_data()
train_dataset, test_dataset = create_splits(dataset)
train_loader = make_loader(hyperparameters['batch_size'], train_dataset)
test_loader = make_loader(hyperparameters['batch_size'], test_dataset)

In [8]:
# simple training loop using the existing `tensor` as toy data
device = 'cuda'
diffusion.to(device)
diffusion.eps_model.train()

optimizer = torch.optim.Adam(diffusion.eps_model.parameters(), lr=1e-4)

epochs = 300
val_interval = 10

with wandb.init(project="diffusion_test", config=hyperparameters):
    wandb.watch(diffusion.eps_model, log="all", log_freq=10)
    for epoch in range(epochs):
        diffusion.set_new_noise_schedule(phase='train')
        train_loss = 0
        for batch in tqdm(train_loader, desc=f'Epoch {epoch+1}/{epochs}'):
            X, y = batch['img_bev_embed'], batch['pts_bev_embed']
            X = X.to(device)
            y = y.to(device)
            loss = diffusion.forward(y0=y, y_cond=X)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
            print(f"Batch Loss: {loss.item()}")
        
        avg_train_loss = train_loss / len(train_loader)
        wandb.log({'train_loss': avg_train_loss}, step=epoch)

    wandb.finish()


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: mln9d4 (mln9d4-tu-delft). Use `wandb login --relogin` to force relogin


Epoch 1/300:   9%|▉         | 1/11 [00:02<00:21,  2.17s/it]

Batch Loss: 0.8441227078437805


Epoch 1/300:  18%|█▊        | 2/11 [00:04<00:17,  1.97s/it]

Batch Loss: 0.8360666632652283


...

Batch Loss: 0.7150452733039856


Epoch 20/300:   0%|          | 0/11 [00:01<?, ?it/s]
Traceback (most recent call last):
  File "/tmp/ipykernel_2964214/603521227.py", line 20, in <module>
    loss = diffusion.forward(y0=y, y_cond=X)
  File "/tmp/ipykernel_2964214/913358632.py", line 133, in forward
    noise_hat = self.eps_model(torch.cat([y_noisy, y_cond], dim=1) if y_cond is not None else y_noisy, sample_gammas)
  File "/home/mingdayang/miniconda3/envs/unibev/lib/python3.7/site-packages/torch/nn/modules/module.py", line 1123, in _call_impl
    hook_result = hook(self, input, result)
  File "/home/mingdayang/miniconda3/envs/unibev/lib/python3.7/site-packages/wandb/integration/torch/wandb_torch.py", line 114, in <lambda>
    mod, inp, outp, log_track_params
  File "/home/mingdayang/miniconda3/envs/unibev/lib/python3.7/site-packages/wandb/integration/torch/wandb_torch.py", line 108, in parameter_log_hook
    self.log_tensor_stats(data.cpu(), "parameters/" + prefix + name)
KeyboardInterrupt


BrokenPipeError: [Errno 32] Broken pipe

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x72415c0d95d0>> (for post_run_cell):


BrokenPipeError: [Errno 32] Broken pipe

In [ ]:
wandb.finish()

In [ ]:
checkpoint = torch.load('checkpoints/diffusion_model_square.pth')

Unet=UNet(**checkpoint['model_config'])
Unet.load_state_dict(checkpoint['model_state_dict'])
diffusion = DenoiseDiffusion(Unet, checkpoint['beta_schedule'], 'cuda:0')

diffusion.set_new_noise_schedule(phase='test')
print(diffusion.eps_model.in_channel)

In [ ]:

diffusion.set_new_noise_schedule(phase='test')
y_ddpm, _ = diffusion.ddpm_sampler(1)
y_ddim, _ = diffusion.ddim_sampler(1, steps=10, eta=0.0)



In [ ]:
start = time.time()
y, _ = diffusion.ddim_sampler(1, steps=30, clip_denoised=True)
end = time.time()
print(y)
print(f"Max: {torch.amax(y)}, Min: {torch.amin(y)}")
print(f"Execution time: {end - start}")
plt.imshow(y.cpu().numpy()[0, 0], cmap='viridis')
plt.colorbar()
plt.show()

In [ ]:
start = time.time()
y, _ = diffusion.ddim_sampler(1, steps=50, clip_denoised=True)
end = time.time()
print(y)
print(f"Max: {torch.amax(y)}, Min: {torch.amin(y)}")
print(f"Execution time: {end - start}")
plt.imshow(y.cpu().numpy()[0, 0], cmap='viridis')
plt.colorbar()
plt.show()

In [ ]:
y_ddpm, _ = diffusion.ddpm_sampler(1)
y_ddim, _ = diffusion.ddim_sampler(1, steps=10)
fig, ax = plt.subplots(1, 2)

im0 = ax[0].imshow(y_ddpm.cpu().numpy()[0, 0], cmap='viridis')
im1 = ax[1].imshow(y_ddim.cpu().numpy()[0, 0], cmap='viridis')

fig.colorbar(im0, ax=ax[0])
fig.colorbar(im1, ax=ax[1])

plt.show()

In [ ]:
def save_model(model, model_config, beta_schedule):
    save_temp = {}
    save_temp['model_state_dict'] = model.eps_model.state_dict()
    save_temp['model_config'] = model_config
    save_temp['beta_schedule'] = beta_schedule
    torch.save(save_temp, 'checkpoints/temp.pth')